### MAC Season 4 Weather Data
- This notebook contains the code used to process raw weather station data from Maricopa Agricultural Center
- Data modified or added
    - sliced for season 4 dates only
    - added dates
    - added cumulative growing degree days
    - added cumulative precipitation
    - added First and Second drought treatment boolean columns
        - `True` or `False` for dates that received first treatment
        - `True` or `False` for dates that received second treatment

In [ ]:
import datetime
import numpy as np
import pandas as pd

In [ ]:
# Read in season 4 weather data from MAC weather station

weather_df_0 = pd.read_csv('data/raw/mac/mac_weather_station_raw_daily_2017.csv')
# print(weather_df_0.shape)
# weather_df_0.head(3)

#### Slice dataframe for season dates only and add date column

- Planting Date: 2017-04-20, Day 110
- Last Day of Harvest: 2017-09-16, Day 259

In [ ]:
weather_df_1 = weather_df_0.loc[(weather_df_0.day_of_year >= 110) & (weather_df_0.day_of_year <= 259)]
# print(weather_df_1.shape)
# weather_df_1.head(3)

In [ ]:
season_4_date_range = pd.date_range(start='2017-04-20', end='2017-09-16')

In [ ]:
weather_df_2 = weather_df_1.copy()
weather_df_2['date'] = season_4_date_range
# print(weather_df_2.shape)
# weather_df_2.tail(3)

#### Add Growing Degree Days
- 10 degrees Celsius is base temp for sorghum
- Daily gdd value = ((max temp + min temp) / 2) - 10 (base temp)
- Growing Degree Days = cumulative sum of daily gdd values
- Negative values convert to 0

In [ ]:
weather_df_3 = weather_df_2.copy()
weather_df_3['daily_gdd'] = (((weather_df_3['air_temp_max'] + weather_df_3['air_temp_min'])) / 2) - 10
# print(weather_df_3.shape)
# weather_df_3.head(3)

In [ ]:
# Check for negative values - will return empty df if there are none

weather_df_3.loc[weather_df_3.daily_gdd < 0]

In [ ]:
weather_df_4 = weather_df_3.copy()
weather_df_4['gdd'] = np.rint(np.cumsum(weather_df_4['daily_gdd']))
# print(weather_df_4.shape)
# weather_df_4.tail(3)

In [ ]:
# Drop daily gdd value, keep only cumulative gdd values

weather_df_5 = weather_df_4.drop(labels='daily_gdd', axis=1)
# print(weather_df_5.shape)
# weather_df_5.tail()

In [ ]:
weather_df_6 = weather_df_5.copy()
weather_df_6['cum_precip'] = np.round(np.cumsum(weather_df_6.precip_total), 2)
# weather_df_6.head(3)

#### Add columns for water-deficit stress treatment 
- First water-deficit stress treatment: 2017-08-01 through 2017-08-14
- Second water-deficit stress treatment: 2017-08-15 through 2017-08-30

In [ ]:
first_treatment_dates = pd.date_range(start='2017-08-01', end='2017-08-14')
second_treatment_dates = pd.date_range(start='2017-08-15', end='2017-08-30')

In [ ]:
season_dates = weather_df_6.date.values

In [ ]:
# If a date falls within a treatment date range, it will have a value of True in the water deficit columns
# False values indicate regular irrigation on that date

first_treatment_col = []


for d in season_dates:
    
    if d in first_treatment_dates:
        
        first_treatment_col.append(True)
        
    else: 
        
        first_treatment_col.append(False)
        
# print(len(first_treatment_col))

In [ ]:
second_treatment_col = []

for d in season_dates:
    
    if d in second_treatment_dates:
        
        second_treatment_col.append(True)
        
    else:
        
        second_treatment_col.append(False)

In [ ]:
weather_df_7 = weather_df_6.copy()

weather_df_7['first_water_deficit_treatment'] = first_treatment_col
weather_df_7['second_water_deficit_treatment'] = second_treatment_col

# print(weather_df_7.shape)
# weather_df_7.head(3)

In [ ]:
weather_df_8 = weather_df_7.rename({'vpd_mean': 'vpd_mean_kpa'}, axis=1)
weather_df_8.head(3)

In [ ]:
weather_df_8.to_csv('data/weather/mac_season_4_weather.csv')